In [1]:
!sudo apt -y install nodejs
!jupyter labextension install jupyter-threejs
!pip install -q -U torchdiffeq




The following additional packages will be installed:
  libc-ares2 libnode64 nodejs-doc
Suggested packages:
  npm
The following NEW packages will be installed:
  libc-ares2 libnode64 nodejs nodejs-doc
0 upgraded, 4 newly installed, 0 to remove and 71 not upgraded.
Need to get 6807 kB of archives.
After this operation, 30.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libc-ares2 amd64 1.15.0-1ubuntu0.4 [36.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libnode64 amd64 10.19.0~dfsg-3ubuntu1.3 [5766 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 nodejs-doc all 10.19.0~dfsg-3ubuntu1.3 [943 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 nodejs amd64 10.19.0~dfsg-3ubuntu1.3 [61.4 kB]
Fetched 6807 kB in 3s (2608 kB/s)

78Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 108782 files and directories currently installed.)
Pre

# Cubes

In [2]:
from pythreejs import *
import ipywidgets, random, math
from IPython.display import display, HTML
from pythreejs._example_helper import use_example_model_ids
from PIL import Image, ImageDraw, ImageFilter
use_example_model_ids()

img = Image.new("RGB", (25,25), (255, 255, 255))
draw = ImageDraw.Draw(img)
draw.line((0, 25), fill=(0,0,0))
for x in range(25):
    for y in range(25):
        img.putpixel((x,y), (random.choice(range(256)),random.choice(range(256)),random.choice(range(256))))
img.filter(ImageFilter.EMBOSS)
img.save('mcraft.jpg')

view_height, view_width = 400, 600
ct = ImageTexture('mcraft.jpg')

colors = ['red','white','blue','green','yellow','orange']
cubes = []
for x in range(3):
    for y in range(3):
        for z in range(3):
            if any([x!=1, y!=1,z!=1]):
                cubes.append(Mesh(BoxBufferGeometry(0.9, 0.9, 0.9), [MeshPhysicalMaterial(color='green', map=ct), MeshPhysicalMaterial(color='yellow'), MeshPhysicalMaterial(color='blue'), MeshPhysicalMaterial(color='red'), MeshPhysicalMaterial(color='white'), MeshPhysicalMaterial(color='orange')], position=[x, y, z]))
                
camera = PerspectiveCamera( position=[5, 5, 5], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()
scene = Scene(children= cubes + [camera, key_light, ambient_light] )
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=view_width, height=view_height)
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(5.0, 5.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

# Wreaths

In [3]:
from itertools import product
def points(cx, cy, cz, r, n):
    points = []
    for i in range(n):
        x = cx + r * math.cos(2 * math.pi * i / n)
        y = cy + r * math.sin(2 * math.pi * i / n)
        z = cz # may need more math for multi dimension z to spin around
        points.append([x, y, z])
    return points

colors = ['red','yellow']
spheres = [Mesh(SphereBufferGeometry(1, 32, 32), MeshStandardMaterial(color=random.choice(colors)), position=(x,y,z)) for x,y,z in points(3, 3, 0, 4, 12)]
colors = ['blue','yellow']
spheres += [Mesh(SphereBufferGeometry(1, 32, 32), MeshStandardMaterial(color=random.choice(colors)), position=(x,y,z)) for x,y,z in points(9, 9, 0, 4, 12)]

camera2 = PerspectiveCamera( position=[20, 20, 20], aspect=view_width/view_height)
key_light2 = DirectionalLight(position=[0, 0, 10])
ambient_light2 = AmbientLight()
controller2 = OrbitControls(controlling=camera2)
scene2 = Scene(children= spheres + [camera2, key_light2, ambient_light2] )
controller2 = OrbitControls(controlling=camera2)
renderer2 = Renderer(camera=camera2, scene=scene2, controls=[controller2], width=view_width, height=view_height)
display(renderer2)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(20.0, 20.0, 20.0), projectionMatrix=(1.0, 0.0, 0.0, 0.…

# Globes

In [4]:
import numpy as np

#code from
#https://pythreejs.readthedocs.io/en/stable/examples/Textures.html
#globe_6/8
arr_w = 6
arr_h = 8

def gen_checkers(width, height, n_checkers_x, n_checkers_y):
    array = np.ones((width, height, 3), dtype='float32')
    # width in texels of each checker
    checker_w = width / n_checkers_x
    checker_h = height / n_checkers_y
    for y in range(height):
        for x in range(width):
            color_key = int(x / checker_w) + int(y / checker_h)
            if color_key % 2 == 0:
                array[x, y, :] = [ 0, 0, 0 ]
            else:
                array[x, y, :] = [ 1, 1, 1 ]
    # We need to flip x/y since threejs/webgl insists on column-major data for DataTexture
    return np.swapaxes(array, 0, 1)
data_tex = DataTexture(data=gen_checkers(arr_w, arr_h, 4, 2), format="RGBFormat", type="FloatType",)
display(data_tex)

Preview(child=DataTexture(data=array([[[0., 0., 0.],
        [0., 0., 0.],
        [1., 1., 1.],
        [0., …

You can handle the rest of the magic from here

In [5]:
#Just a few move string changes
import pandas as pd
pd.set_option('mode.chained_assignment', None)

#https://www.kaggle.com/code/crodoc/greedy-improvement-of-valid-solutions
sub = pd.read_csv('/kaggle/input/greedy-improvement-of-valid-solutions/submission.csv')

sub['moves'] = sub['moves'].map(lambda x: x.split('.'))

sub.moves[151] = sub.moves.values[151][:45] +  ['d1', 'f2', 'f2']  + sub.moves.values[151][52:]
sub.moves[155] = sub.moves.values[155][:52] +  ['r3', 'r2', 'r2']  + sub.moves.values[155][57:]
sub.moves[156] = sub.moves.values[156][:42] +  ['f2', 'f2', 'f3']  + sub.moves.values[156][47:]
sub.moves[162] = sub.moves.values[162][:44] +  ['d1', 'f2', 'f2']  + sub.moves.values[162][51:]
sub.moves[168] = sub.moves.values[168][:59] +  ['f2', 'f2', 'd0']  + sub.moves.values[168][66:]
sub.moves[168] = sub.moves.values[168][:37] +  ['f2', 'f2', 'f3']  + sub.moves.values[168][42:]

sub.moves[171] = sub.moves.values[171][:14] +  ['f2', 'f2', 'f3']  + sub.moves.values[171][19:]
sub.moves[172] = sub.moves.values[172][:43] +  ['f2', 'f2', 'd0']  + sub.moves.values[172][50:]
sub.moves[172] = sub.moves.values[172][:22] +  ['d1', 'f2', 'f2']  + sub.moves.values[172][29:]
sub.moves[173] = sub.moves.values[173][:29] +  ['d1', 'f2', 'f2']  + sub.moves.values[173][36:]

sub.moves[174] = sub.moves.values[174][:52] +  ['r2', 'r2', 'd3']  + sub.moves.values[174][59:]
sub.moves[175] = sub.moves.values[175][:32] +  ['d2', 'd2', 'f3']  + sub.moves.values[175][39:]
sub.moves[178] = sub.moves.values[178][:43] +  ['f2', 'f2', 'f3']  + sub.moves.values[178][48:]
sub.moves[179] = sub.moves.values[179][:46] +  ['f2', 'f2', 'f3']  + sub.moves.values[179][51:]
sub.moves[182] = sub.moves.values[182][:46] +  ['f2', 'f2', 'f3']  + sub.moves.values[182][51:]
sub.moves[182] = sub.moves.values[182][:36] +  ['d1', 'f2', 'f2']  + sub.moves.values[182][43:]
sub.moves[183] = sub.moves.values[183][:56] +  ['d0', 'r2', 'r2']  + sub.moves.values[183][63:]
sub.moves[184] = sub.moves.values[184][:58] +  ['f2', 'f2', 'r0']  + sub.moves.values[184][65:]
sub.moves[192] = sub.moves.values[192][:39] +  ['r2', 'r2', 'f0']  + sub.moves.values[192][46:]
sub.moves[194] = sub.moves.values[194][:36] +  ['d0', 'r2', 'r2']  + sub.moves.values[194][43:]
sub.moves[196] = sub.moves.values[196][:50] +  ['f0', 'f2', 'f2']  + sub.moves.values[196][57:]
sub.moves[198] = sub.moves.values[198][:57] +  ['d3', 'r2', 'r2']  + sub.moves.values[198][64:]
sub.moves[199] = sub.moves.values[199][:47] +  ['f0', 'r2', 'r2']  + sub.moves.values[199][54:]
sub.moves[199] = sub.moves.values[199][:30] +  ['f0', 'd2', 'd2']  + sub.moves.values[199][37:]
sub.moves[205] = sub.moves.values[205][:35] +  ['r0', 'f2', 'd3']  + sub.moves.values[205][44:]
sub.moves[212] = sub.moves.values[212][:51] +  ['f3', 'f3', 'f4']  + sub.moves.values[212][56:]
sub.moves[217] = sub.moves.values[217][:80] +  ['r3', 'r3', 'r4']  + sub.moves.values[217][85:]
sub.moves[217] = sub.moves.values[217][:28] +  ['f4', 'd3', 'd3']  + sub.moves.values[217][35:]
sub.moves[218] = sub.moves.values[218][:66] +  ['d3', 'd3', 'r0']  + sub.moves.values[218][73:]
sub.moves[221] = sub.moves.values[221][:85] +  ['r1', 'r1', 'd0']  + sub.moves.values[221][92:]
sub.moves[223] = sub.moves.values[223][:34] +  ['d3', 'd3', 'd4']  + sub.moves.values[223][39:]
sub.moves[228] = sub.moves.values[228][:41] +  ['d0', 'f3', 'f3']  + sub.moves.values[228][48:]
sub.moves[231] = sub.moves.values[231][:40] +  ['f3', 'f3', 'f4']  + sub.moves.values[231][45:]
sub.moves[233] = sub.moves.values[233][:28] +  ['d4', 'd3', 'd3']  + sub.moves.values[233][33:]

sub['moves'] = sub['moves'].map(lambda x: '.'.join(x))
sub.to_csv('submission.csv', index=False)
sub.head()

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1


# Ｈ𝐀𝑷𝑷𝓎 🇰𝗮𝘨𝘨🇱𝖎Ｎɢ 💯